In [25]:
using DataFrames
using Plots
using MLTools

df = readtable("train.csv");

# Variablen-Typen
Für statistische Variablen lassen sich gewisse Kenngrössen bestimmen welche die 
**allgemeine/durchschnittliche Tendenz**, die **Streuung** oder die **Form** der zugrundeliegenden
Verteilung beschreiben. Je nach dem welche von diesen Grössen für den konkreten Fall (=Variable)
wohldefiniert sind, kann man zwischen 3 Haupttypen unterscheiden.

### Allgemeine Tendenz
- Arithmetisches Mittel: $\bar{x} = \frac{1}{n}\sum_{i=1}^n x_i$
- Median: Es gibt $\frac{n}{2}$ Werte die Grösser sind als der Median und $\frac{n}{2}$ die kleiner sind.
- Mode: Der am häufigsten vorkommende Wert.

### Streuung
- Gesamter Wertebereich (min bis max)
- Quantile (die ersten, zweiten, letzten.. x-ten Prozent)
- Varianz: $\sigma^2 = \frac{1}{n}\sum_{i=1}^n (x_i - \bar{x})^2$
- Standardabweichung: $\sigma = \sqrt{\sigma^2}$

### Form
- Skewness (Schiefe): Wie symmetrisch ist die Verteilung? $\frac{1}{n}\sum_{i=1}^n (x_i - \bar{x})^3$
- Kurtosis (Wölbung): Wie spitzig ist die Verteilung $\frac{1}{n}\sum_{i=1}^n (x_i - \bar{x})^4$

# Quantitative Variablen
Lassen Berechnungen für alle oben genannten Kenngrössen zu und
werden in der Regel durch Reale Zahlen dargestellt, wie z.B:
- Körpergewicht in kg
- Temperatur in Grad Kelvin
- Kokainverbrauch pro Person pro Jahr
- Anzahl Einwohner

Weiter liegt Quantitativen Variablen eine sinnvolle numerische Skala zugrunde auf der sich eine
"Distanz" berechnen lässt. Als Beispiel ist es sinnvoll zu sagen 2000 kg sind 4 mal so viel wie 500 kg.

**Alle oben genannten Kenngrössen sind wohldefiniert.**

# Ordinale Variablen
Haben keine sinnvolle nuerische Skala als Grundlage, es gibt aber eine eindeutig definierte Reihenfolge der Elemente. Man kann also Elemente sortieren, aber keine sinnvolle euklidische Distanz zwischen ihen Bestimmen.
- Hausnummern
- T-shirt Grössen
- Akunto Abo-Preise

**Definierte Kenngrössen: Reihenfolge, Median, Mode, Quantile**

# Kategorische Variablen
Es lassen sich weder eine Reihenfolge, noch eine Distanz zwischen Elementen sinnvoll definieren.

- Farben
- Vornamen
- Nationalität

**Definierte Kenngrössen: Mode**






In [26]:
# Untersuche nochmals die Daten und Bestimme den Typ für jede Variable:
names(df)

12-element Array{Symbol,1}:
 :PassengerId
 :Survived   
 :Pclass     
 :Name       
 :Sex        
 :Age        
 :SibSp      
 :Parch      
 :Ticket     
 :Fare       
 :Cabin      
 :Embarked   

In [27]:
head(df)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NA,S
2,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
3,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.925,NA,S
4,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1,C123,S
5,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.05,NA,S
6,6,0,3,"Moran, Mr. James",male,NA,0,0,330877,8.4583,NA,Q


# Fehlende Daten

Werden in Julia (DataFrames) mit NA gekennzeichnet. Obwohl NAs in DataFrames erlaubt sind, so dürfen wir keine fehlenden Daten in unsere Machine Learning Methoden einbauen. Die Daten sind ja genau der Schlüssel zu einem guten Modell.

Im einfachsten Fall entfernen wir einfach alle Messungen mit NAs. In Julia geht das mit der Funktion **complete_cases!(df)**, welche nur Zeilen ohne NAs beibehält. In der Regel wollen wir aber so viele Daten wie möglich behalten, um nicht potentielle Information in diesen Zeilen wegzuwerfen.

Wir müssen also "von Hand" eigene Werte für die NAs einfügen, am besten solche die möglichst sinnvoll sind. Es gibt allerdings kein Standardverfahren um dieses Problem zu lösen, denn es hängt vom jeweiligen Problem ab, was wir als sinnvoll erachten.

Eine oft verwendete Strategie ist das ersetzen der NAs mit einer der Grössen **mean**, **mode** oder **median**.


In [7]:
# Untersuche die Variable :Embarked auf NAs und beantworte folgende Fragen:
# Tipp: Die folgenden Funktionen sind nützlich:
# isna(df[:Embarked])
# df[:Embarked] .== "S"
# sum()

# 1) Wieviele NAs kommen vor?

# 2) Mean, Mode oder Median als "Füller"?

# 3) ML-Modell aus den restlichen Variablen um :Embarked vorherzusagen?

# 4) Können wir so ein Modell (mit unserem jetzigen Wissen) bauen?

In [10]:
# Dasselbe Spiel mit der Variablen :Age

# One Hot Encoding
Mit one hot encoding kann man Kategorische Variablen in numerische Werte umwandeln. Dabei werden neue **Dummy-Variablen** eingeführt. Eine neue Variable für jeden in der ursprünglichen Variable vorkommenden Wert. Diese Variablen werden dann auf 0 oder 1 gesetzt wenn der entsprechende Wert vorkommt.

Im :Embarked Beispiel werden also neu die Dummy-Variablen S, Q, und C eingeführt.
Dies kann durch die Funktion **onehot!(df, :Embarked)** erreicht werden

In [21]:
onehot!(df, :Embarked)
head(df)

LoadError: TypeError: non-boolean (DataArrays.NAtype) used in boolean context

# Binarization
Ähnlich müssen wir bei der Variable "Sex" vorgehen. Da nur zwei verschiedene Werte vorkommen werden wir diese mit der Funktion **binarize!(df, :Sex)** 1 und 0 ersetzen. Es ist hier nicht nötig (sogar hinderlich) zwei Variablen "male" und "female" einzuführen welche beide jeweils 1 und 0 für die entsprechenden Werte enthalten. 

Dieselbe Menge an Information kann und soll in einer einzigen Variable ausgedrückt werden. Die zweite Variable ist im Grunde nur die inverse der ersten, es besteht also eine direkte korrelation zwischen diesen beiden Variablen. Für die Machine-Learning Algorithmen ist dann nur schwer beurteilbar welche Performance-Änderungen bei der Suche nach dem Optimum von der einen oder der anderen Variablen kommt.

**Auch im One Hote Encoding fall ist es ratsam eine Variable wegzulassen**, da wiederum diesselbe Menge and Information durch N-1 binäre Variablen wiedergegeben werden kann.

In [28]:
binarize!(df, :Sex)
head(df)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.25,NA,S
2,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",1,38.0,1,0,PC 17599,71.2833,C85,C
3,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.925,NA,S
4,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1,C123,S
5,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.05,NA,S
6,6,0,3,"Moran, Mr. James",0,NA,0,0,330877,8.4583,NA,Q


# Machine Learning Modell
Wir können nun ein Machine-Learning Modell aufstellen um das Überleben der Passagiere vorauszusagen.
Dazu konvertieren wir Variablen aus dem DataFrame in reguläre Julia-Arrays.
Die unabhängigen Variablen werden X genannt (Matrix), die abhängige Variable y (Vektor)
Dazu brauchen wir die **convert()** Funktion:

convert(Matrix, df[:, [var1, var2, var3... varN])

convert(Vector, df[:, var]

wobei var1 etc die zu verwendenden Variablen sind

In [17]:
# X = convert(Matrix, df[:, [:Pclass, :Age]])
# y = ...

# bei X muss dann noch eine Spalte mit Einsen angefügt werden, der sogenannte Offset oder Bias Term.
# die Funktion hcat(a, b) kann dabei Vektoren oder Matrizen horizontal zusammenfügen.
# Mit ones(N) wird ein Vektor mit N Einsen gebildet

# Schliesslich muss noch der Vektor theta initialisiert werden.
# Die Funktion randn(N) erzeugt einen Vektor mit N normalverteilten Zufallszahlen.

In [ ]:
model = LogisticRegression(X, y, theta)
result = MLTools.fit!(model)
prediction = MLTools.predict(X, model)